Generate trip plan report

# Trip Report

## Configure OpenAI Settings

In [10]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT", "").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Azure AI Search (Cognitive vector store)
vector_store_address: str = os.getenv("AAG_AZURE_SEARCH_SERVICE_ENDPOINT")
vector_store_password: str = os.getenv("AAG_AZURE_SEARCH_ADMIN_KEY")
# index_name: str = "langchain-vector-arxiv-physics"

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

## Azure OpenAI ChatCompletion

### Read finds from previous processed data

In [11]:
file_path = './reports/skitripplanning_prompt.txt'

try:
    with open(file_path, 'r') as file:
        content = file.read()
        print(content)
except FileNotFoundError:
    print("The file doesn't exist or the path is incorrect.")



I am going to plan a Ski SLC, UT ski trip for a week. Here are the information on this trip:
1. I have booked airport parking: The parking information is as follows:
{
  "address": "14707 Guy R Brewer Boulevard, New York, United States",
  "rating": "4",
  "reviews": "539",
  "type": "Commercial Parking Lot or Garage",
  "price": "$7.50 per day",
  "check-in": "Wed, Dec 06, 09:00 AM",
  "check-out": "Tue, Dec 12, 11:30 PM"
}
2. Booked flight: The flight information is as follows:
{
  "flight1": {
    "departure": "New York-JFK",
    "arrival": "Salt Lake City",
    "date": "Dec 6, 2023",
    "departureTime": "11:00 am",
    "arrivalTime": "02:15 pm",
    "duration": "5 hrs 15 mins",
    "stops": "Nonstop",
    "class": "Blue Basic",
    "flightNumber": "B6871",
    "operatedBy": "JetBlue"
  },
  "flight2": {
    "departure": "Salt Lake City",
    "arrival": "New York-JFK",
    "date": "Dec 12, 2023",
    "departureTime": "03:11 pm",
    "arrivalTime": "09:38 pm",
    "duration": "4 hr

### Run ChatCompletion with user prompt to generate trip reports

In [12]:
prompt_text = content
# print(prompt_text)

response = openai.ChatCompletion.create(
    engine=DEPLOYMENT_NAME_CHAT,
    messages=[
        {"role": "system", "content": "You are an expert level travel agent."},
        {"role": "user", "content": prompt_text},
    ],
    max_tokens=15000,
)
report_content = response["choices"][0]["message"]["content"]

print(response["choices"][0]["message"]["content"])

Here is the comprehensive 7-day ski trip itinerary:

Day 1: Departure
Date: Dec 6, 2023
Arrive at the airport. Airport Parking Info: 
- Address: 14707 Guy R Brewer Boulevard, New York, United States
- Rating: 4
- Reviews: 539
- Type: Commercial Parking Lot or Garage
- Price: $7.50 per day
- Check-in: Wed, Dec 06, 09:00 AM
- Check-out: Tue, Dec 12, 11:30 PM
Park your car at the pre-booked parking spot.
Check-in for your flight. Flight Info: 
- Departure: New York-JFK
- Arrival: Salt Lake City
- Date: Dec 6, 2023
- Departure Time: 11:00 am
- Arrival Time: 02:15 pm
- Duration: 5 hrs 15 mins
- Stops: Nonstop
- Class: Blue Basic
- Flight Number: B6871
- Operated By: JetBlue
Depart for your destination.
Upon arrival, pick up your pre-booked rental car. Car Rental Info: 
- Pick-up: Wed, Dec 6
- Drop-off: Tue, Dec 12
- Pick-up and Drop-off Instructions: Location
- Confirmation: #WF********
- Expedia Itinerary: 7267**********
Drive to your hotel.
Hotel Contact Info: 
- Hotel Name: Fairfield Inn

In [13]:
report_content

'Here is the comprehensive 7-day ski trip itinerary:\n\nDay 1: Departure\nDate: Dec 6, 2023\nArrive at the airport. Airport Parking Info: \n- Address: 14707 Guy R Brewer Boulevard, New York, United States\n- Rating: 4\n- Reviews: 539\n- Type: Commercial Parking Lot or Garage\n- Price: $7.50 per day\n- Check-in: Wed, Dec 06, 09:00 AM\n- Check-out: Tue, Dec 12, 11:30 PM\nPark your car at the pre-booked parking spot.\nCheck-in for your flight. Flight Info: \n- Departure: New York-JFK\n- Arrival: Salt Lake City\n- Date: Dec 6, 2023\n- Departure Time: 11:00 am\n- Arrival Time: 02:15 pm\n- Duration: 5 hrs 15 mins\n- Stops: Nonstop\n- Class: Blue Basic\n- Flight Number: B6871\n- Operated By: JetBlue\nDepart for your destination.\nUpon arrival, pick up your pre-booked rental car. Car Rental Info: \n- Pick-up: Wed, Dec 6\n- Drop-off: Tue, Dec 12\n- Pick-up and Drop-off Instructions: Location\n- Confirmation: #WF********\n- Expedia Itinerary: 7267**********\nDrive to your hotel.\nHotel Contact I

In [9]:
word_file = './reports/ai_Ski_trip_planning_001.txt'

with open(word_file, 'w') as file:
    file.write(report_content)